# EMR on EC2 - Account Usage Report (Monthly)

This notebook provides a breakdown of the costs for EMR on EC2 Clusters running in your AWS account. The dashboard generated at the end of the notebook provides the following information:

- Aggregated hourly costs for all the clusters running in the account
- AWS Costs repartitioning by service usage (Amazon EC2, Amazon EMR, EBS Storage, Data Transfer)
- Total Costs for all EMR clusters running in the account
- Instance Trends to determine which instances are launched by the EMR service (Instance type, family, market, architecture)

To generate the analytics cost and usage dashboard, please modify the following **Configuration** section to match your environment, and run this notebook.

***
## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* This notebook uses the `PySpark` kernel.
* The EMR cluster attached to this notebook should have `Spark` `JupyterEnterpriseGateway` `Livy` applications installed.
* The EMR cluster should be configured to use the [AWS Glue Data Catalog as metastore for Spark](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-spark-glue.html).
***
## Configuration

In [ ]:
# Current year and month used to filter data
from datetime import date
current_date = date.today()
spark.conf.set("year", current_date.year)
spark.conf.set("month", current_date.month)

# AWS Glue Database where the AWS Cost and Usage Report is located (e.g. athenacurcfn_a_w_s_costs_daily)
spark.conf.set("cur_db", "YOUR_CUR_DATABASE_NAME")

# AWS Glue Table Name used by AWS Cost and Usage Report (e.g. aws_costs_daily)
spark.conf.set("cur_table", "YOUR_CUR_TABLE_NAME")

### Data collection

In [ ]:
%%sql -o emr_detailed_hourly_costs -n -1 -q
SELECT 
  split_part(line_item_resource_id, '/', 2) AS cluster, 
  line_item_usage_account_id AS account,
  date_format(line_item_usage_start_date - INTERVAL 1 hours , 'yyyy-MM-dd HH:mm') AS date,
  year(line_item_usage_start_date - INTERVAL 1 hours ) AS year, 
  month(line_item_usage_start_date - INTERVAL 1 hours ) AS month, 
  day(line_item_usage_start_date - INTERVAL 1 hours ) AS day, 
  hour(line_item_usage_start_date - INTERVAL 1 hours ) AS hour, 
  product_product_name AS service, 
  product_product_family AS family, 
  sum(line_item_unblended_cost) AS cost 
FROM ${cur_db}.${cur_table}
WHERE 
  product_product_name = 'Amazon Elastic MapReduce' 
  AND year = '${year}'
  AND month = '${month}'
  AND split_part(line_item_resource_id, '/', 2) NOT IN ('null', 'applications', 'virtualclusters') 
  AND split_part(line_item_resource_id, '/', 2) <> '' 
GROUP BY 
  split_part(line_item_resource_id, '/', 2), 
  line_item_usage_account_id,
  line_item_usage_start_date,
  year(line_item_usage_start_date), 
  month(line_item_usage_start_date), 
  day(line_item_usage_start_date), 
  hour(line_item_usage_start_date), 
  product_product_name, 
  product_product_family 
UNION ALL 
SELECT 
  resource_tags_aws_elasticmapreduce_job_flow_id AS cluster, 
  line_item_usage_account_id AS account,
  date_format(line_item_usage_start_date, 'yyyy-MM-dd HH:mm') AS date,
  year(line_item_usage_start_date) AS year, 
  month(line_item_usage_start_date) AS month, 
  day(line_item_usage_start_date) AS day, 
  hour(line_item_usage_start_date) AS hour, 
  product_product_name AS service, 
  product_product_family AS family, 
  sum(line_item_unblended_cost) AS cost 
FROM ${cur_db}.${cur_table}
WHERE 
  product_product_name = 'Amazon Elastic Compute Cloud' 
  AND year = '${year}'
  AND month = '${month}'
  AND resource_tags_aws_elasticmapreduce_job_flow_id NOT IN ('null', 'applications', 'virtualclusters') 
  AND resource_tags_aws_elasticmapreduce_job_flow_id <> '' 
GROUP BY 
  resource_tags_aws_elasticmapreduce_job_flow_id, 
  line_item_usage_account_id,
  line_item_usage_start_date,
  year(line_item_usage_start_date), 
  month(line_item_usage_start_date), 
  day(line_item_usage_start_date), 
  hour(line_item_usage_start_date), 
  product_product_name, 
  product_product_family 

In [ ]:
%%sql -o emr_instance_details -n -1 -q
SELECT DISTINCT
line_item_resource_id AS instance_id,
product_physical_processor AS arch,
product_instance_type_family AS instance_family,
product_instance_family AS instance_detail,
product_instance_type AS instance_type,
product_marketoption AS market,
1 as num
FROM ${cur_db}.${cur_table}
WHERE 
  product_product_name = 'Amazon Elastic Compute Cloud' 
  AND year = '${year}'
  AND month = '${month}'
  AND product_product_family = 'Compute Instance'
  AND resource_tags_aws_elasticmapreduce_job_flow_id NOT IN ('null', 'applications', 'virtualclusters') 
  AND resource_tags_aws_elasticmapreduce_job_flow_id <> ''
  AND product_instance_type_family <> ''


### Dashboard

In [ ]:
%%local
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Default Theme
pio.templates["aws_template"] = go.layout.Template(
    layout_autosize=True,
    layout_height=600
)
pio.templates.default = 'aws_template+gridon'

# Cluster Cost Details
account_hourly_split = emr_detailed_hourly_costs[['date', 'family', 'cost']].groupby(['date', 'family'], as_index=False)['cost'].sum()
account_month_split = emr_detailed_hourly_costs[['month', 'family', 'cost']].groupby(['month', 'family'], as_index=False)['cost'].sum()
account_cluster_costs = emr_detailed_hourly_costs[['cluster', 'year', 'month', 'cost']].groupby(['cluster', 'year', 'month'], as_index=False)['cost'].sum().sort_values(by='cost', ascending=False)

## Hourly Cost Dashboard
fig = px.bar(account_hourly_split, x=account_hourly_split.date, y=account_hourly_split.cost, color=account_hourly_split.family)
fig.update_layout(title='Hourly Costs', xaxis_title="", yaxis_title="Costs ($)")
fig.update_layout(legend_title_text='', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

## Costs Trends
fig = make_subplots(rows=1, cols=3, specs=[[{"type": "xy"}, {"type": "domain"},{"type": "table"}]], subplot_titles=("Expensive Clusters - Top 10", "Split by Service", "All Cluster Costs"))
fig.add_trace(go.Bar(x=account_cluster_costs.cluster, y=account_cluster_costs.head(10).cost.round(2), text=account_cluster_costs.cost.round(2), texttemplate = "%{y:$,.2f}", showlegend=False, name=""), row=1, col=1)
fig.add_trace(go.Pie(labels=account_month_split.family, values=account_month_split.cost.round(2), name="", texttemplate = "<b>%{label}</b>: %{value:$,.2f} <br> %{percent}", textposition = "inside"), row=1, col=2)
fig.add_trace(go.Table(header=dict(values=['Cluster', 'Costs ($)'], align="left"), cells=dict(values=[account_cluster_costs.cluster, account_cluster_costs.cost.round(2)], align='left')), row=1, col=3)

fig.update_layout(height=500, showlegend=False, title='Costs Trends')
fig.update_yaxes(title_text="Costs ($)", row=1, col=1)
fig.show()

## Instance Trends
instance_market = emr_instance_details[['market', 'num']].groupby(['market'], as_index=False)['num'].sum()
instance_family = emr_instance_details[['instance_family', 'instance_type', 'num']].groupby(['instance_family', 'instance_type'], as_index=False)['num'].sum()
instance_arch = emr_instance_details[['arch', 'num']].groupby(['arch'], as_index=False)['num'].sum()
instance_desc = emr_instance_details[['instance_detail', 'num']].groupby(['instance_detail'], as_index=False)['num'].sum()

fig = px.bar(instance_family, x=instance_family.instance_type, y=instance_family.num)
fig.update_layout(title='Instance Trends', xaxis_title="", yaxis_title="Usage")
fig.update_layout(legend_title_text='', legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=1))
fig.show()

fig = make_subplots(rows=1, cols=3, specs=[[{"type": "domain"}, {"type": "domain"}, {"type": "domain"}]], subplot_titles=("Processor", "Market", "Families"))
fig.add_trace(go.Pie(labels=instance_arch.arch, values=instance_arch.num, name="", texttemplate = "<b>%{label}</b>: %{percent}", textposition = "inside"), row=1, col=1)
fig.add_trace(go.Pie(labels=instance_market.market, values=instance_market.num, name="", texttemplate = "<b>%{label}</b>: %{percent}", textposition = "inside"), row=1, col=2)
fig.add_trace(go.Pie(labels=instance_desc.instance_detail, values=instance_desc.num, name="", texttemplate = "<b>%{label}</b>: %{percent}", textposition = "inside"), row=1, col=3)
fig.update_layout(height=500, showlegend=False)
fig.show()